# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 1 2023 8:30AM,256346,10,0086342323,ISDIN Corporation,Released
1,Feb 1 2023 8:30AM,256346,10,0086342333,ISDIN Corporation,Released
2,Feb 1 2023 8:30AM,256346,10,0086343365,ISDIN Corporation,Released
3,Feb 1 2023 8:30AM,256333,10,0086343366,ISDIN Corporation,Released
4,Feb 1 2023 8:30AM,256333,10,0086343369,ISDIN Corporation,Released
5,Feb 1 2023 8:30AM,256333,10,0086343370,ISDIN Corporation,Released
6,Feb 1 2023 8:30AM,256333,10,0086343371,ISDIN Corporation,Released
7,Feb 1 2023 8:30AM,256333,10,0086343372,ISDIN Corporation,Released
8,Feb 1 2023 8:30AM,256333,10,0086343368,ISDIN Corporation,Released
9,Feb 1 2023 8:30AM,256333,10,0086343367,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,256338,Released,1
46,256340,Released,1
47,256342,Released,15
48,256344,Released,1
49,256346,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256338,NaN,NaN,1.0
256340,NaN,NaN,1.0
256342,NaN,NaN,15.0
256344,NaN,NaN,1.0
256346,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256240,25.0,0.0,12.0
256259,6.0,3.0,3.0
256264,37.0,52.0,5.0
256270,0.0,2.0,0.0
256271,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256240,25,0,12
256259,6,3,3
256264,37,52,5
256270,0,2,0
256271,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256240,25,0,12
1,256259,6,3,3
2,256264,37,52,5
3,256270,0,2,0
4,256271,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256240,25,,12
1,256259,6,3,3
2,256264,37,52,5
3,256270,,2,
4,256271,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 1 2023 8:30AM,256346,10,ISDIN Corporation
3,Feb 1 2023 8:30AM,256333,10,ISDIN Corporation
22,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation
43,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc."
44,Feb 1 2023 8:30AM,256337,10,"Citieffe, Inc."
45,Jan 31 2023 4:00PM,256344,12,"SD Head USA, LLC"
46,Jan 31 2023 3:46PM,256340,16,Sartorius Bioprocess Solutions
47,Jan 31 2023 3:22PM,256336,22,"NBTY Global, Inc."
48,Jan 31 2023 2:51PM,256330,10,MedStone Pharma LLC
53,Jan 31 2023 2:50PM,256329,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 1 2023 8:30AM,256346,10,ISDIN Corporation,,,11
1,Feb 1 2023 8:30AM,256333,10,ISDIN Corporation,,,17
2,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation,,,15
3,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",,,1
4,Feb 1 2023 8:30AM,256337,10,"Citieffe, Inc.",,1,
5,Jan 31 2023 4:00PM,256344,12,"SD Head USA, LLC",,,1
6,Jan 31 2023 3:46PM,256340,16,Sartorius Bioprocess Solutions,,,1
7,Jan 31 2023 3:22PM,256336,22,"NBTY Global, Inc.",,,1
8,Jan 31 2023 2:51PM,256330,10,MedStone Pharma LLC,,1,4
9,Jan 31 2023 2:50PM,256329,10,ISDIN Corporation,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 8:30AM,256346,10,ISDIN Corporation,11,,
1,Feb 1 2023 8:30AM,256333,10,ISDIN Corporation,17,,
2,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation,15,,
3,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",1,,
4,Feb 1 2023 8:30AM,256337,10,"Citieffe, Inc.",,1,
5,Jan 31 2023 4:00PM,256344,12,"SD Head USA, LLC",1,,
6,Jan 31 2023 3:46PM,256340,16,Sartorius Bioprocess Solutions,1,,
7,Jan 31 2023 3:22PM,256336,22,"NBTY Global, Inc.",1,,
8,Jan 31 2023 2:51PM,256330,10,MedStone Pharma LLC,4,1,
9,Jan 31 2023 2:50PM,256329,10,ISDIN Corporation,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 8:30AM,256346,10,ISDIN Corporation,11,,
1,Feb 1 2023 8:30AM,256333,10,ISDIN Corporation,17,,
2,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation,15,,
3,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",1,,
4,Feb 1 2023 8:30AM,256337,10,"Citieffe, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 8:30AM,256346,10,ISDIN Corporation,11.0,NaN,NaN
1,Feb 1 2023 8:30AM,256333,10,ISDIN Corporation,17.0,NaN,NaN
2,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation,15.0,NaN,NaN
3,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Feb 1 2023 8:30AM,256337,10,"Citieffe, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 8:30AM,256346,10,ISDIN Corporation,11.0,0.0,0.0
1,Feb 1 2023 8:30AM,256333,10,ISDIN Corporation,17.0,0.0,0.0
2,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation,15.0,0.0,0.0
3,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Feb 1 2023 8:30AM,256337,10,"Citieffe, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4869844,86.0,19.0,7.0
12,256344,1.0,0.0,0.0
15,768776,23.0,52.0,65.0
16,1281561,3.0,2.0,0.0
18,512598,2.0,0.0,0.0
19,1281451,21.0,9.0,0.0
20,256300,1.0,0.0,0.0
21,512633,2.0,0.0,0.0
22,256336,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4869844,86.0,19.0,7.0
1,12,256344,1.0,0.0,0.0
2,15,768776,23.0,52.0,65.0
3,16,1281561,3.0,2.0,0.0
4,18,512598,2.0,0.0,0.0
5,19,1281451,21.0,9.0,0.0
6,20,256300,1.0,0.0,0.0
7,21,512633,2.0,0.0,0.0
8,22,256336,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,86.0,19.0,7.0
1,12,1.0,0.0,0.0
2,15,23.0,52.0,65.0
3,16,3.0,2.0,0.0
4,18,2.0,0.0,0.0
5,19,21.0,9.0,0.0
6,20,1.0,0.0,0.0
7,21,2.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,86.0
1,12,Released,1.0
2,15,Released,23.0
3,16,Released,3.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,7.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,19.0,0.0,52.0,2.0,0.0,9.0,0.0,0.0,0.0
Released,86.0,1.0,23.0,3.0,2.0,21.0,1.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,7.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,52.0,2.0,0.0,9.0,0.0,0.0,0.0
2,Released,86.0,1.0,23.0,3.0,2.0,21.0,1.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,7.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,52.0,2.0,0.0,9.0,0.0,0.0,0.0
2,Released,86.0,1.0,23.0,3.0,2.0,21.0,1.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()